Translations using random seed

In [13]:
import pandas as pd
import json
import os
# with open("test.fr-en", 'rb') as file:
#     str_file = file.read()
#     print(str_file)
#     json_file = json.load(str_file.decode('utf-8'))
#     print("json_file", json_file)
#     # print(pd.DataFrame(json.loads(file.read())))

In [2]:
import os

In [20]:
import requests
# resp = requests.get("https://datasets-server.huggingface.co/first-rows?dataset=wmt14&config=fr-en&split=test") # Only first rows
resp = requests.get("https://datasets-server.huggingface.co/rows?dataset=wmt14&config=fr-en&split=test&offset=0&length=3000")
json_data = resp.json()

In [25]:
 # Get user's home directory
import os
from datasets import load_dataset
home = os.path.expanduser("~")

# Define the path of the cache directory
cache_dir = os.path.join(home, ".cache", "huggingface", "datasets")

# Define the name and configuration of the dataset
dataset_name = "wmt14"
config_name = "fr-en"

# Build the path for the specific dataset configuration
dataset_config_path = os.path.join(cache_dir, dataset_name, config_name)

print(f"Checking cache at: {dataset_config_path}")

# Check if the dataset configuration is already cached
if os.path.exists(dataset_config_path) and len(os.listdir(dataset_config_path)) > 0:
    print("Dataset already downloaded, loading from cache.")
    # If the dataset is already downloaded, load it from the cache directory
    dataset = load_dataset(dataset_name, config_name, cache_dir=cache_dir)
else:
    print("Downloading the dataset.")
    # Download the dataset and specify the cache directory
    dataset = load_dataset(dataset_name, config_name, cache_dir=cache_dir)

# Here, you should adjust the loading of subsets to avoid redundant downloads or loading.
# Load complete Test Dataset
test_dataset = dataset["test"]


/home/paperspace/anaconda3/envs/preprocess_bert_udem/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checking cache at: /home/paperspace/.cache/huggingface/datasets/wmt14/fr-en
Dataset already downloaded, loading from cache.


/home/paperspace/anaconda3/envs/preprocess_bert_udem/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for wmt14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wmt14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [26]:
test_dataset

Dataset({
    features: ['translation'],
    num_rows: 3003
})

In [5]:
# json_data['rows'][0]['row']['translation']['fr']

In [28]:
# json_data['rows'][0]
def get_json_data(data):    
    
    en = list()
    fr = list()
    for element in data["translation"]:
        # print("element: ", element)
        en.append(element["en"])
        fr.append(element["fr"])
    return en, fr

In [29]:
en, fr = get_json_data(test_dataset)

In [30]:
len(en), len(fr)

(3003, 3003)

In [9]:
# fr

In [31]:
json_data_df = pd.DataFrame(list(zip(en,fr)), columns=['src','target'])

In [11]:
# json_data_df.iloc[0]

In [32]:
import deep_translator
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source="auto", target='fr')

Translations on random indices of src

In [33]:
# json_data_df['ht_mt_target']
import numpy as np
import random
from random import sample
random.seed(12345)

# given data frame df

# create random index
rindex =  np.array(sample(range(len(json_data_df)), int((len(json_data_df)/2))))

print("rindex: ", rindex)
# get 10 random rows from df
# dfr = df.ix[rindex]

rindex:  [1706 3001   41 ...  250 1946 1632]


In [34]:
json_data_df.head(10)

,src,target
0,Spectacular Wingsuit Jump Over Bogota,"Spectaculaire saut en ""wingsuit"" au-dessus de ..."
1,Sportsman Jhonathan Florez jumped from a helic...,Le sportif Jhonathan Florez a sauté jeudi d'un...
2,"Wearing a wingsuit, he flew past over the famo...",Equipé d'un wingsuit (une combinaison munie d'...
3,A black box in your car?,Une boîte noire dans votre voiture ?
4,As America's road planners struggle to find th...,Alors que les planificateurs du réseau routier...
5,"The devices, which track every mile a motorist...","Les appareils, qui enregistrent tous les miles..."
6,The usually dull arena of highway planning has...,Le secteur généralement sans intérêt de la pla...
7,Libertarians have joined environmental groups ...,Les libertaires ont rejoint des groupes écolog...
8,The tea party is aghast.,Le Tea Party est atterré.
9,The American Civil Liberties Union is deeply c...,L'American Civil Liberties Union est elle auss...


In [35]:
def translate_with_max_length(text, max_length=128): # Need to change the mac_length = 50
    # Translate the text to French
    translated_text = translator.translate(text)

    # Check if the translated text is within the desired maximum length
    if len(translated_text) <= max_length:
        return translated_text
    else:
        # Truncate the translated text to the maximum length
        return translated_text[:max_length]

In [11]:
# Iterate through DataFrame rows
for i, row in json_data_df.iterrows():
    if i in rindex:
        # Translate the 'src' column and limit to max_len 50
        translated_text = translate_with_max_length(row['src'], max_length=128)
        json_data_df.loc[i, 'ht_mt_target'] = translated_text
        json_data_df.loc[i, 'ht_mt_label'] = '0'
    else:
        # Use the original 'target' column
        json_data_df.loc[i, 'ht_mt_target'] = row['target']
        json_data_df.loc[i, 'ht_mt_label'] = '1'

In [ ]:
# json_data_df.ix[rindex] 
# for row in json_data_df.ix[] 
# for row in rindex:
#     print("json_data_df[row] ", json_data_df.iloc[row])
    # if json_data_df.ix[row]
    # print("row: ", row)
    
# for row in rindex:
#     json_data_df.ix[row]
# for i, row in json_data_df.iterrows():
#     if i in rindex:
#         json_data_df.loc[i, 'ht_mt_target'] = translator.translate(json_data_df.loc[i, 'src'])
#         json_data_df.loc[i, 'ht_mt_label'] = '0'
#     else:
#         json_data_df.loc[i, 'ht_mt_target'] = json_data_df.loc[i, 'target']
#         json_data_df.loc[i, 'ht_mt_label'] = '1'

        

In [ ]:
json_data_df

,src,target,ht_mt_target,ht_mt_label
0,Spectacular Wingsuit Jump Over Bogota,"Spectaculaire saut en ""wingsuit"" au-dessus de ...",Saut spectaculaire en Wingsuit au-dessus de Bo...,0
1,Sportsman Jhonathan Florez jumped from a helic...,Le sportif Jhonathan Florez a sauté jeudi d'un...,Le sportif Jhonathan Florez a sauté jeudi d'un...,0
2,"Wearing a wingsuit, he flew past over the famo...",Equipé d'un wingsuit (une combinaison munie d'...,Equipé d'un wingsuit (une combinaison munie d'...,1
3,A black box in your car?,Une boîte noire dans votre voiture ?,Une boîte noire dans votre voiture ?,0
4,As America's road planners struggle to find th...,Alors que les planificateurs du réseau routier...,Alors que les planificateurs du réseau routier...,1
...,...,...,...,...
95,It is therefore not surprising that he should ...,Rien d'étonnant à ce qu'il ait donc un masque ...,Il n'est donc pas étonnant qu'il porte un masq...,0
96,"He appears in one of their adverts, broadcast ...",Il apparaîtra dans un de leurs spots publicita...,Il apparaîtra dans un de leurs spots publicita...,1
97,"The Minister of Defence, Rob Nicholson, insist...","Le ministre de la Défense, Rob Nicholson, a in...","Le ministre de la Défense, Rob Nicholson, a in...",1
98,Attacked by liberals and neo-democrats in the ...,Attaqué par les libéraux et les néo-démocrates...,Attaqué par les libéraux et les néo-démocrates...,1


In [ ]:
idx = np.where((json_data_df['ht_mt_label']=='0'))

In [ ]:
json_data_df.loc[idx]

,src,target,ht_mt_target,ht_mt_label
0,Spectacular Wingsuit Jump Over Bogota,"Spectaculaire saut en ""wingsuit"" au-dessus de ...",Saut spectaculaire en Wingsuit au-dessus de Bo...,0
1,Sportsman Jhonathan Florez jumped from a helic...,Le sportif Jhonathan Florez a sauté jeudi d'un...,Le sportif Jhonathan Florez a sauté jeudi d'un...,0
3,A black box in your car?,Une boîte noire dans votre voiture ?,Une boîte noire dans votre voiture ?,0
9,The American Civil Liberties Union is deeply c...,L'American Civil Liberties Union est elle auss...,L'Union américaine des libertés civiles est ég...,0
10,And while Congress can't agree on whether to p...,Et tandis que les membres du Congrès n'arriven...,Et même si le Congrès ne parvient pas à se met...,0
11,"They are exploring how, over the next decade, ...","Ils cherchent comment, au cours de la prochain...","Ils étudient comment, au cours de la prochaine...",0
15,There is going to be a change in how we pay th...,Il va y avoir du changement dans la façon dont...,Il va y avoir un changement dans la façon dont...,0
18,Americans don't buy as much gas as they used to.,Les Américains n'achètent plus autant d'essenc...,Les Américains n’achètent plus autant d’essenc...,0
20,"The federal tax itself, 18.4 cents per gallon,...","La taxe fédérale elle-même, qui est de 18,4 ce...","La taxe fédérale elle-même, 18,4 cents le gall...",0
21,Politicians are loath to raise the tax even on...,Les hommes politiques sont réticents à augment...,Les politiciens sont réticents à augmenter la ...,0


Writing files to test_classify/...

In [14]:
if not os.path.exists('test_data/wmt14_en_fr_test/standard_classifiers'):
    os.makedirs('test_data/wmt14_en_fr_test/standard_classifiers')
output_path = 'test_data/wmt14_en_fr_test/standard_classifiers/'

In [16]:
json_data_df = json_data_df.drop(columns=['target'])

In [17]:
# write the df to csv file
json_data_df.to_csv(output_path + 'wmt14_en_fr_test_std_classifiers.csv', index=False)

In [ ]:
# with open(os.path.join(output_path, "ht_mt_target.fr"), 'w') as json_to_txt_v:
#     for row in json_data_df['ht_mt_target'].values:
#         json_to_txt_v.writelines(row)
#         json_to_txt_v.write("\n")#.encode('utf-8')")

In [ ]:
# with open(os.path.join(output_path, "src.en"), 'w') as json_to_txt_v:
#     for row in json_data_df['src'].values:
#         json_to_txt_v.writelines(row)
#         json_to_txt_v.write("\n")#.encode('utf-8')")

In [ ]:
# with open(os.path.join(output_path,"target.fr"), 'w') as json_to_txt_v:
#     for row in json_data_df['target'].values:
#         json_to_txt_v.writelines(row)
#         json_to_txt_v.write("\n")#.encode('utf-8')")

In [ ]:
# with open(os.path.join(output_path,"ht_mt_label"), 'w') as json_to_txt_v:
#     for row in json_data_df['ht_mt_label']:
#         json_to_txt_v.write(str(row))
#         json_to_txt_v.write("\n")#.encode('utf-8')")